In [1]:
import cv2

img = cv2.imread('a.png')

In [2]:
img.shape

(1620, 2160, 3)

In [15]:
# col = [[30,440], [445,855], [860, 1270], [1275, 1685], [1690, 2100]]
# row = [[100, 510], [515, 925], [930, 1340]]
col = [[43,535], [548,1040], [1058, 1550], [1563, 2055], [2068, 2560]]
row = [[98, 590], [608, 1100], [1113, 1605]]

In [16]:
row, col

([[98, 590], [608, 1100], [1113, 1605]],
 [[43, 535], [548, 1040], [1058, 1550], [1563, 2055], [2068, 2560]])

In [24]:
# for col_idx in col:
#     for row_idx in row:
#         sub = img[col_idx[0]:col_idx[1], row_idx[0]:row_idx[1]]

def clip2(img, prefix_list, prefix_idx=0, start_idx=1, out_dir='./temp/'):
    for i in range(15):
        c = int(i/3)
        r = i%3
        perfix=prefix_list[prefix_idx]

        col_idx = col[c] 
        row_idx = row[r]
        sub = img[row_idx[0]:row_idx[1], col_idx[0]:col_idx[1]]
        sub = cv2.resize(sub, (410, 410), interpolation=cv2.INTER_CUBIC)
        
        cv2.imwrite(out_dir+perfix[c]+'_'+str(start_idx+r).zfill(5)+'.png', sub)

img1 = cv2.imread('../original_dataset/cheng/13.png')
img2 = cv2.imread('../original_dataset/cheng/14.png')
start_idx = 82
prefix_list = [['U0031', 'U0032', 'U0033', 'U0034', 'U0035'], ['U0036', 'U0037', 'U0038', 'U0039', 'U0030']]
clip2(img1, prefix_list, prefix_idx=0, start_idx=start_idx)
clip2(img2, prefix_list, prefix_idx=1, start_idx=start_idx)

## real solution

In [3]:
import cv2
import numpy as np

threshold = 200000

def clean_arr(arr):
    pre = arr[0]
    arr_clean = [pre]

    for i in arr:
        if np.abs(i-pre) < 5:
            pre = i
            continue
        else:
            arr_clean.append(i)
            pre = i
        
    return arr_clean

def clip(img, prefix_list, prefix_idx=0, start_idx=1, out_dir='./temp/'):
    # cv2默认为bgr顺序
    c_b = np.where(img.sum(0)[:,1] < threshold)
    c_g = np.where(img.sum(0)[:,0] < threshold)
    r_b = np.where(img.sum(1)[:,1] < threshold)
    r_g = np.where(img.sum(1)[:,0] < threshold)

    row = np.intersect1d(r_b,r_g)
    col = np.intersect1d(c_b,c_g)

    row_clean = clean_arr(row)
    col_clean = clean_arr(col)

    for i in range(15):
        c = int(i/3)
        r = i%3
        perfix=prefix_list[prefix_idx]
        sub = img[row_clean[r]+3:row_clean[r+1]-2, col_clean[c]+3:col_clean[c+1]-2]
        cv2.imwrite(out_dir+perfix[c]+'_'+str(start_idx+r).zfill(5)+'.png', sub)
#         cv2.imwrite(out_dir+'out'+str(i)+'.png', sub)

In [4]:
img1 = cv2.imread('../original_dataset/cheng/01.png')
img2 = cv2.imread('../original_dataset/cheng/02.png')
start_idx = 64
prefix_list = [['U0031', 'U0032', 'U0033', 'U0034', 'U0035'], ['U0036', 'U0037', 'U0038', 'U0039', 'U0030']]
clip(img1, prefix_list, prefix_idx=0, start_idx=start_idx)
clip(img2, prefix_list, prefix_idx=1, start_idx=start_idx)

IndexError: list index out of range

# change csv file

In [1]:
def change_csv(original_file, out_file, new_writer):
    with open(original_file) as f:
        lines = [x for x in f.readlines()]
        
    new_list = []

    for line in lines[1:]:
#         import pdb
#         pdb.set_trace()
        new_name = str(int(line.split('\"')[1].split('_')[1].split('.')[0])+63).zfill(5)+'.png"'
        new_line = line.split(',')
        new_line[0] = new_line[0].split('_')[0]+'_'+new_name
        new_line[2] = new_writer
        new_line = "".join(new_line).replace(' ', ', ')
        new_list.append(new_line)
        
    with open(out_file, 'w') as f:
        for i in new_list:
            f.write(i)

In [2]:
new_writer = ' "Cheng"\n'
original_file = '../dataset_clip/shen.csv'
out_file = '../dataset_clip/cheng.csv'
change_csv(original_file, out_file, new_writer)

In [87]:
new_lines = lines[1].split(',')
new_lines[0] = new_lines[0].split('_')[0]+'_'+new_name
new_lines[2] = new_writer
len(new_lines)
# new_lines.pop(2)
new_lines

['"U0030/U0030_00022.png', ' "Pencil"', ' "Wang"\n']

In [89]:
new_lines = "".join(new_lines)
new_lines.replace(' ', ', ')

'"U0030/U0030_00022.png, "Pencil", "Wang"\n'